## 주유소 정보 페이지 띄우기

In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

url = "https://www.opinet.co.kr/searRgSelect.do"
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.set_window_size(1400, 1000)

driver.get(url)

## 서울시 내 구 데이터부터 찾기

In [45]:
from selenium.webdriver.common.by import By

sido = driver.find_element(By.ID, "SIDO_NM0")
sido_list = sido.find_elements(By.TAG_NAME, "option")
for idx, sido_name in enumerate(sido_list):
    print(str(idx) + ". " + sido_name.get_attribute("value"))

0. 
1. 서울특별시
2. 부산광역시
3. 대구광역시
4. 인천광역시
5. 광주광역시
6. 대전광역시
7. 울산광역시
8. 세종특별자치시
9. 경기도
10. 강원특별자치도
11. 충청북도
12. 충청남도
13. 전북특별자치도
14. 전라남도
15. 경상북도
16. 경상남도
17. 제주특별자치도


In [46]:
sido_names = [sido_name.get_attribute("value") for sido_name in sido_list]
sido_names

['',
 '서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [47]:
sido_names = sido_names[1:]
sido_names

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [48]:
print(sido_names[0])
sido.send_keys(sido_names[0])

서울특별시


In [49]:
gu = driver.find_element(By.ID, "SIGUNGU_NM0")
gu_list = gu.find_elements(By.TAG_NAME, "option")

gu_names = [gu_name.get_attribute("value") for gu_name in gu_list]
print(len(gu_names), gu_names)

26 ['', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


In [50]:
gu_names.remove('')
print(len(gu_names), gu_names)

25 ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


## 특정 구의 주유소 팝업창 띄우기

In [51]:
gu.send_keys(gu_names[12])

In [52]:
oil_list = driver.find_element(By.ID, "body1")
station = oil_list.find_elements(By.TAG_NAME, "a")

station_temp = station[2]
station_temp.click()

## 팝업창에서 정보 추출

In [53]:
popup = driver.find_element(By.CLASS_NAME, "atlanmap-info-content")

name = popup.find_element(By.ID, "os_nm") # 주유소 이름
phone_num = popup.find_element(By.ID, "phn_no") # 주유소 전화번호
addr = popup.find_element(By.ID, "rd_addr") # 주유소 주소
brand = popup.find_element(By.ID, "poll_div_nm") # 주유소 상표

# 유가정보
price_table = popup.find_element(By.ID, "infoTbody")
price_cont = price_table.find_elements(By.TAG_NAME, "tr")
price_info = {}
for line in price_cont:
    oil_type = line.find_elements(By.TAG_NAME, "td")[0].text # 유종
    oil_price = line.find_elements(By.TAG_NAME, "td")[1].text # 가격

    price_info[oil_type] = oil_price

# 부가정보
service = popup.find_element(By.CLASS_NAME, "service")
service_img = service.find_elements(By.TAG_NAME, "img")
service_list = []
for img in service_img:
    # image 이름 기준으로 부가 서비스 유무 파악
    if "off" not in img.get_attribute("src"):
        service_list.append(img.get_attribute("alt"))

# 각 정보 출력
print(name.text)
print(phone_num.text)
print(addr.text)
print(brand.text)

for key, value in price_info.items():
    print(key, value)

print(service_list)

성산대교셀프주유소
02-323-0078
서울 마포구 성산로 144
HD현대오일뱅크
 
보통휘발유 1,679원
경유 1,589원
실내등유 1,649원
['세차장', '경정비']


## 특정 구 안에서 정보 추출 테스트

In [57]:
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

oil_list = driver.find_element(By.ID, "body1")
stations = oil_list.find_elements(By.TAG_NAME, "a")

name_list = []
phn_list = []
addr_list = []
brand_list = []
price_list = []
service_list = []

for station in stations:
    station.click()

    popup = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, "atlanmap-info-content")))

    name = popup.find_element(By.ID, "os_nm").text # 주유소 이름
    phone_num = popup.find_element(By.ID, "phn_no").text # 주유소 전화번호
    addr = popup.find_element(By.ID, "rd_addr").text # 주유소 주소
    brand = popup.find_element(By.ID, "poll_div_nm").text # 주유소 상표

    # 유가정보
    price_table = popup.find_element(By.ID, "infoTbody")
    price_cont = price_table.find_elements(By.TAG_NAME, "tr")
    price_info = {}
    for line in price_cont:
        oil_type = line.find_elements(By.TAG_NAME, "td")[0].text # 유종
        oil_price = line.find_elements(By.TAG_NAME, "td")[1].text # 가격

        price_info[oil_type] = oil_price

    # 부가정보
    service = popup.find_element(By.CLASS_NAME, "service")
    service_img = service.find_elements(By.TAG_NAME, "img")
    services = []
    for img in service_img:
        # image 이름 기준으로 부가 서비스 유무 파악
        if "off" not in img.get_attribute("src"):
            services.append(img.get_attribute("alt"))

    name_list.append(name)
    phn_list.append(phone_num)
    addr_list.append(addr)
    brand_list.append(brand)
    price_list.append(price_info)
    service_list.append(services)


In [58]:
# 리스트 길이 확인
len(name_list), len(phn_list), len(addr_list), len(brand_list), len(price_list), len(service_list)

(12, 12, 12, 12, 12, 12)

In [67]:
# dataframe으로 변환
import pandas as pd

# 리스트를 문자열로 변환하는 함수
def clean_list_to_string(x):
    if isinstance(x, list):
        return ", ".join(x)  # 리스트의 요소들을 쉼표로 구분하여 문자열로 변환
    return x

oil_data = {'상호':name_list, 
            '전화번호':phn_list, 
            "주소":addr_list,
            "상표":brand_list,
            "유종별 가격":price_list,
            "부가정보":service_list}

oil_df = pd.DataFrame(oil_data)

oil_df.sort_index(inplace=True)
oil_df['부가정보'] = oil_df['부가정보'].apply(clean_list_to_string)

oil_df

,상호,전화번호,주소,상표,유종별 가격,부가정보
0,(주)타이거통상 월드컵주유소,070-4121-0021,서울 마포구 월드컵북로 170 (성산제2동),S-OIL,"{'고급휘발유': '1,889원', '보통휘발유': '1,659원', '경유': '...",
1,마포시엠주유소,02-338-9447,서울 마포구 월드컵북로 62,SK에너지,"{'': '', '보통휘발유': '1,669원', '경유': '1,549원'}",
2,성산대교셀프주유소,02-323-0078,서울 마포구 성산로 144,HD현대오일뱅크,"{'': '', '보통휘발유': '1,679원', '경유': '1,589원', '실...","세차장, 경정비"
3,청원주유소,02-325-5533,서울 마포구 월드컵북로 113 (성산동),HD현대오일뱅크,"{'고급휘발유': '1,978원', '보통휘발유': '1,679원', '경유': '...",세차장
4,(주)안국에너지 망원동주유소,02-334-0917,서울 마포구 월드컵로 119 (망원동),SK에너지,"{'': '', '보통휘발유': '1,758원', '경유': '1,628원'}",세차장
5,용강주유소,070-7795-4800,서울 마포구 토정로 258,S-OIL,"{'': '', '보통휘발유': '1,779원', '경유': '1,669원', '실...",
6,에쓰오일(주) 염리동주유소,02-3272-5155,서울 마포구 백범로 126 (염리동),S-OIL,"{'': '', '보통휘발유': '1,779원', '경유': '1,669원'}",세차장
7,마포주유소,02-711-5189,서울 마포구 마포대로 186,S-OIL,"{'고급휘발유': '1,942원', '보통휘발유': '1,792원', '경유': '...",세차장
8,에스에스오토셀프,02-336-5185,서울 마포구 서강로 76 (창전동),HD현대오일뱅크,"{'고급휘발유': '2,098원', '보통휘발유': '1,799원', '경유': '...","세차장, 24시영업"
9,(주)만성상사 대흥주유소,02-3273-5151,서울 마포구 대흥로 61 (대흥동),SK에너지,"{'고급휘발유': '2,199원', '보통휘발유': '1,799원', '경유': '...","세차장, 24시영업"


In [68]:
oil_df = pd.concat([oil_df.drop(columns=['유종별 가격']), oil_df['유종별 가격'].apply(pd.Series)], axis=1)
oil_df

,상호,전화번호,주소,상표,부가정보,고급휘발유,보통휘발유,경유,,실내등유
0,(주)타이거통상 월드컵주유소,070-4121-0021,서울 마포구 월드컵북로 170 (성산제2동),S-OIL,,"1,889원","1,659원","1,539원",,NaN
1,마포시엠주유소,02-338-9447,서울 마포구 월드컵북로 62,SK에너지,,NaN,"1,669원","1,549원",,NaN
2,성산대교셀프주유소,02-323-0078,서울 마포구 성산로 144,HD현대오일뱅크,"세차장, 경정비",NaN,"1,679원","1,589원",,"1,649원"
3,청원주유소,02-325-5533,서울 마포구 월드컵북로 113 (성산동),HD현대오일뱅크,세차장,"1,978원","1,679원","1,589원",,NaN
4,(주)안국에너지 망원동주유소,02-334-0917,서울 마포구 월드컵로 119 (망원동),SK에너지,세차장,NaN,"1,758원","1,628원",,NaN
5,용강주유소,070-7795-4800,서울 마포구 토정로 258,S-OIL,,NaN,"1,779원","1,669원",,"1,650원"
6,에쓰오일(주) 염리동주유소,02-3272-5155,서울 마포구 백범로 126 (염리동),S-OIL,세차장,NaN,"1,779원","1,669원",,NaN
7,마포주유소,02-711-5189,서울 마포구 마포대로 186,S-OIL,세차장,"1,942원","1,792원","1,682원",,NaN
8,에스에스오토셀프,02-336-5185,서울 마포구 서강로 76 (창전동),HD현대오일뱅크,"세차장, 24시영업","2,098원","1,799원","1,689원",,NaN
9,(주)만성상사 대흥주유소,02-3273-5151,서울 마포구 대흥로 61 (대흥동),SK에너지,"세차장, 24시영업","2,199원","1,799원","1,689원",,"1,745원"


In [61]:
driver.quit()